In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mimg
from pathlib import Path
import random


In [ ]:
def crop_coord(img, iter_d, iter_e, iter_o, inter_c):
    '''
    Recortar imagen utilizando tecnicas de preprocesamiento
    '''

    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(7,7),0)
    _, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh = cv2.dilate(thresh, None, iterations = iter_d)
    thresh = cv2.erode(thresh, None, iterations = iter_e)
    kernel = np.ones((7, 7), np.uint8)
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=iter_o)
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=iter_c)

    edges = cv2.Canny(closed,107,255)
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    x, y, w, h = cv2.boundingRect(c)
    

    return x,y,w,h
    
'''Probaré con los dos números para ver qué recortes se realizaron de forma incorrecta y veré la proporción de este nuevo dataset con el original. 
   Si no se han perdido muchos entonces emplearé ese.
   También usaré el de 6 para comparar la precisión entre ambos
'''

def check_best_crop(img):
    width, height = 0
    x,y = 0
    #Se itera hasta que el valor de iteraciones llegue a 40 o bien se cumpla la otra condición. Esto es para encontrar el mejor crop, que diga el recorte
    iterations = 0
    while w < 225 and h < 225 and iterations < 40:
        dilate = random.randint(4,12)
        erode = random.randint(4,12)
        opened = random.randint(1,4)
        closed = random.randint(1,4)

        x_pos, y_pos, w, h, = crop_coord(img, iter_d = dilate, iter_e = erode, iter_o = opened, iter_c = closed)
        width = w
        height = h
        x = x_pos
        y = y_pos
        iterations+= 1
        if iterations >= 40:
            return
    return img[y:y+height,x:x+width]